In [1]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash 
pwd

/content/Template_Project


In [3]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)

In [15]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
from torchvision import transforms
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"], CNN=True)

train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, test_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)
learn = Learner(train_ds, setup_config)
learn.fit(1000)


100%|██████████| 187/187 [00:00<00:00, 1205.32it/s]


epoch: 1/1000	[valid_acc: 0.8981	valid_loss: 0.7136	train_loss: 1.3196]
inf inf
False
nan
0


100%|██████████| 187/187 [00:00<00:00, 1126.44it/s]


epoch: 2/1000	[valid_acc: 0.9349	valid_loss: 0.3857	train_loss: 0.5181]
inf 0.38573507502436
False
inf
new best: valid_loss 0.38573507502436
0


100%|██████████| 187/187 [00:00<00:00, 1195.77it/s]


...

epoch: 7/1000	[valid_acc: 0.9703	valid_loss: 0.1270	train_loss: 0.1318]
0.14472418045376073 0.12704349106008356
False
0.017680689393677174
0


100%|██████████| 187/187 [00:00<00:00, 1188.00it/s]


epoch: 8/1000	[valid_acc: 0.9733	valid_loss: 0.1138	train_loss: 0.1160]
0.14472418045376073 0.11377302087086408
False
0.030951159582896653
0


 81%|████████  | 608/750 [00:02<00:00, 297.67it/s]


KeyboardInterrupt: 

In [24]:
for key, val in learn.learn.history_raw.items()[1:]:
    print(key)

TypeError: 'dict_items' object is not subscriptable

In [169]:
sample = next(iter(test_dl))
imgs, targets = sample

In [170]:
_, pred = torch.max(learn.learn.model(imgs.unsqueeze(1).to("cuda")).data, 1)
print(sum(pred == targets.to("cuda")))


tensor(64, device='cuda:0')


In [10]:
vars(learn.cbh.EarlyStopping)

{'track_best_vals': {'valid_acc': [<ufunc 'greater'>, 0.986129679144385],
  'valid_loss': [<ufunc 'less'>, 0.05486499901143784],
  'train_loss': [<ufunc 'less'>, 0.049315743035326404]},
 'monitor': 'valid_loss',
 'patience': 1,
 'counter': 0,
 'tmp_history': {},
 'delta': 0.1,
 'learn': <core.dl_framework.learner.Container at 0x7fb1db62beb8>,
 'comp': <ufunc 'less'>,
 'best_val': 0.06441440603093189,
 'epoch': 19,
 'loss': tensor(0.0547, device='cuda:0', grad_fn=<NllLossBackward0>)}

In [7]:
# !pip install plotly
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# def print_history(history):
history = learn.history

implemented_metric = ["acc", "loss"]
plot_dict = {}
for met in implemented_metric:
    plot_dict[met] = [s for s in history if met in s]

fig = make_subplots(rows=1, cols=len(plot_dict), subplot_titles=list(plot_dict.keys()))

for idx, (met, mets) in enumerate(plot_dict.items()):
    for key in mets:
        fig.add_trace(go.Scatter(x=history.index, y=history[key].values, name=key), row=1, col=idx+1 )
    # break
# learn.history.keys()[0]
# fig.update_layout(title_text="test")
fig.show()

In [17]:
list(plot_dict.keys())

['acc', 'loss']

In [22]:
globals()["test_class"]


__main__.test_class

In [29]:
%%timeit
test_num.max()

1.57 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
%%timeit
max(test_dict["test"])


281 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [47]:
import types
class test_class():
    def __init__(self, a):
        self.attr = a
    
    def attr(self):
        pass

test = test_class(1)

getattr(test_class, "attr")

<function __main__.test_class.attr(self)>

In [35]:
%%timeit
test1.sum()

2.09 µs ± 52.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [32]:
import numpy as np

In [16]:
class test1:
    def __init__(self, a):
        self.a = a
    def test_func(self):
        return self.a
a = test1(1)
setattr(a, "s", 123)
vars(a)

{'a': 1, 's': 123}

In [17]:
monitor = Monitor_Cb(["val_acc"])
early = EarlyStopping_Cb(monitor="val_loss")
recorder = Recorder_Cb("hi")
cbh = CallbackHandler([monitor, early, recorder])
cbh.Monitor_Cb.learn


NameError: name 'Monitor_Cb' is not defined

In [9]:
a = ["a"]
for i in a[:4]:
    print(i)

a
